**[ 개요 ]**

- 본 과정은 '기상청허브-계절관측' 데이터를 탐색 및 정제하여, 관측지점 및 연도를 기준으로 각 사계절의 계절별 일수/비율 변화 추이를 살펴볼 수 있는 대시보드용 데이터 생성에 목적을 둔 전처리 과정입니다.
- '1년' 단위는 태양력이 아닌 '겨울 + 봄 + 여름 + 가을' 4계절을 기준으로 합니다. 
- 각 계절에 대응되는 대표 월(月)운 아래와 같이 규정합니다.
    - 겨울 : 전년도 12월, 1월, 2월
    - 봄 : 3월, 4월, 5월
    - 여름 : 6월, 7월, 8월
    - 가을 : 9월, 10월. 11월

## 1. 데이터로드 및 확인

### 1-1) 데이터 로드

In [ ]:
import pandas as pd

# 주피터노트북에서 출력 가능한 최대 레코드 수 조정
pd.set_option('display.max_rows', 100)

In [2]:
def load_kma_data(filepath):
    """KMA 계절 데이터를 로드하고 검증"""
    
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # 1. 헤더 찾기
    data_start = None
    columns = None
    
    for idx, line in enumerate(lines):
        if line.strip().startswith('#') and 'YY' in line and 'STN' in line:
            columns = line.strip().lstrip('#').split()
            data_start = idx + 1
            break
    
    if data_start is None:
        raise ValueError("데이터 헤더를 찾을 수 없습니다.")
    
    # 2. 데이터 추출 
    data_lines = []
    skipped_lines = []
    
    for idx, line in enumerate(lines[data_start:], start=data_start):
        stripped = line.strip()
        # 빈 줄이나 주석 제외
        if not stripped or stripped.startswith('#'):
            continue
        
        clean_parts = stripped.replace(',', '').split()
        # 컬럼 수와 일치하는지 확인
        if len(clean_parts) == len(columns):
            data_lines.append(clean_parts)
        else:
            # 건너뛴 행 기록: (실제 줄 번호, 원본 문자열, 분리된 필드 개수)
            skipped_lines.append((idx + 1, line.strip(), len(clean_parts)))
            print(f"🚨 SKIP LINE {idx + 1} | Expected {columns_len}, Found {len(clean_parts)} | Data: {line.strip()[:60]}...")
    
    # 3. DataFrame 생성
    df = pd.DataFrame(data_lines, columns=columns)

    df['YY'] = df['YY'].astype(int)
    df['STN'] = df['STN'].astype(int)
    df['SSN_ID'] = df['SSN_ID'].astype(int)
    df['SSN_MD'] = df['SSN_MD'].astype(int)
    df['TM'] = pd.to_datetime(df['TM'], errors='coerce')

    return df

In [3]:
file_path = 'KMA_seasonal_data_19741101_20251028.txt'
df = load_kma_data(file_path)

df

,YY,STN,TM,SSN_ID,SSN_MD
0,1974,90,1974-11-03,401,401
1,1974,90,1975-03-21,401,402
2,1974,90,1975-03-31,402,402
3,1974,90,1974-12-03,403,401
4,1974,90,1975-03-23,403,402
...,...,...,...,...,...
111692,2025,247,2025-04-25,503,501
111693,2025,247,2025-05-02,503,504
111694,2025,272,2025-10-20,501,501
111695,2025,279,2025-10-27,501,501


### 1-2) 원시데이터 기본 검토

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111697 entries, 0 to 111696
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   YY      111697 non-null  int64         
 1   STN     111697 non-null  int64         
 2   TM      111697 non-null  datetime64[ns]
 3   SSN_ID  111697 non-null  int64         
 4   SSN_MD  111697 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 4.3 MB


In [5]:
df.isnull().sum()

YY        0
STN       0
TM        0
SSN_ID    0
SSN_MD    0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(0)

### 1-3) 관측지점 & 연도별 레코드 수 확인

In [7]:
df_pivot = df.pivot_table(
    index='STN',
    columns='YY',
    aggfunc='size',
    fill_value=0
)

df_pivot

YY,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
STN,,,,,,,,,,,,,,,,,,,,,
52,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
90,5,37,37,44,44,44,44,44,44,44,...,2,2,2,2,2,2,2,2,2,2
93,0,0,0,0,0,0,0,0,0,0,...,14,45,44,43,45,45,45,45,45,35
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,3,17,15,17,18,23,20,20,17,14,...,2,2,2,2,2,2,2,2,2,2
101,6,38,31,23,34,39,39,37,38,29,...,29,0,0,0,0,0,0,0,0,0
102,0,0,0,0,0,0,0,0,0,0,...,34,36,37,38,39,40,37,38,38,26


## 2. 추가정보 데이터 결합

### 2-1) 계절관측(SSN_ID)

In [8]:
data_id = {
    'SSN_ID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 201, 202, 203, 204, 205, 206, 207, 208, 301, 302, 401, 402, 403, 404, 405, 501, 502, 503],
    'ID_Name': ['제비', '뱀', '개구리', '나비', '잠자리', '종다리', '뻐꾸기', '매미', '기러기', '코스모스', '매화', '개나리', '진달래', '벚나무', '아까시나무', '복숭아', '배나무', '은행나무', '단풍나무', '서리', '얼음', '눈', '관설', '강하전', '유명산 단풍', '해수욕장', '군락단지'],
    'ID_Class': ['동물', '동물', '동물', '동물', '동물', '동물', '동물', '동물', '동물', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(단풍)', '식물(단풍)', '기후', '기후', '기후', '기후', '기후', '식물', '생활', '식물']
}
df_id_map = pd.DataFrame(data_id)
df_id_map

,SSN_ID,ID_Name,ID_Class
0,101,제비,동물
1,102,뱀,동물
2,103,개구리,동물
3,104,나비,동물
4,105,잠자리,동물
5,106,종다리,동물
6,107,뻐꾸기,동물
7,108,매미,동물
8,109,기러기,동물
9,201,코스모스,식물(꽃)


In [9]:
merged_ssn_id = pd.merge(
    df, 
    df_id_map, 
    on='SSN_ID', 
    how='left'
)
merged_ssn_id.head(3)

,YY,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class
0,1974,90,1974-11-03,401,401,서리,기후
1,1974,90,1975-03-21,401,402,서리,기후
2,1974,90,1975-03-31,402,402,얼음,기후


### 2-2) 계절현상(SSN_MD)

In [10]:
data_md = {
    'SSN_MD': [101, 102, 103, 104, 201, 202, 203, 301, 302, 303, 304, 305, 401, 402, 501, 502, 503],
    'MD_Name': ['초견', '중견', '초성', '종성', '발아', '꽃 핌(개화)', '활짝 핌(만발)', '단풍 시작', '단풍 절정', '단풍 끝', '낙엽 시작', '낙엽 끝', '첫(강·하천 결빙)', '마지막(강·하천 해빙)', '시작(해수욕장 개장)', '끝(해수욕장 폐장)', '절정'],
    'MD_Class': ['동물', '동물', '동물', '동물', '식물(꽃)', '식물(꽃)', '식물(꽃)', '식물(단풍)', '식물(단풍)', '식물(단풍)', '식물(단풍)', '식물(단풍)', '기후', '기후', '생활', '생활', '식물']
}
df_md_map = pd.DataFrame(data_md)
df_md_map

,SSN_MD,MD_Name,MD_Class
0,101,초견,동물
1,102,중견,동물
2,103,초성,동물
3,104,종성,동물
4,201,발아,식물(꽃)
5,202,꽃 핌(개화),식물(꽃)
6,203,활짝 핌(만발),식물(꽃)
7,301,단풍 시작,식물(단풍)
8,302,단풍 절정,식물(단풍)
9,303,단풍 끝,식물(단풍)


In [11]:
merged_ssn_nd = pd.merge(
    merged_ssn_id, 
    df_md_map, 
    on='SSN_MD', 
    how='left',
    suffixes=('_ID', '_MD')
)

merged_ssn_nd.head(3)

,YY,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class
0,1974,90,1974-11-03,401,401,서리,기후,첫(강·하천 결빙),기후
1,1974,90,1975-03-21,401,402,서리,기후,마지막(강·하천 해빙),기후
2,1974,90,1975-03-31,402,402,얼음,기후,마지막(강·하천 해빙),기후


### 2-3) 지상관측지점(STN)

In [12]:
import io
data_string = """STN,STN_ko
90,속초
93,북춘천
95,철원
98,동두천
99,파주
100,대관령
101,춘천
102,백령도
104,북강릉
105,강릉
106,동해
108,서울
112,인천
114,원주
115,울릉도
119,수원
121,영월
127,충주
129,서산
130,울진
131,청주
133,대전
135,추풍령
136,안동
137,상주
138,포항
140,군산
143,대구
146,전주
152,울산
155,창원
156,광주
159,부산
162,통영
165,목포
168,여수
169,흑산도
170,완도
172,고창
174,순천
177,홍성
181,서청주
184,제주
185,고산
188,성산
189,서귀포
192,진주
201,강화
202,양평
203,이천
211,인제
212,홍천
216,태백
217,정선군
221,제천
226,보은
232,천안
235,보령
236,부여
238,금산
239,세종
243,부안
244,임실
245,정읍
247,남원
248,장수
251,고창군
252,영광군
253,김해시
254,순창군
255,북창원
257,양산시
258,보성군
259,강진군
260,장흥
261,해남
262,고흥
263,의령군
264,함양군
266,광양시
268,진도군
271,봉화
272,영주
273,문경
276,청송군
277,영덕
278,의성
279,구미
281,영천
283,경주시
284,거창
285,합천
288,밀양
289,산청
294,거제
295,남해
296,북부산
"""

df_stn_map = pd.read_csv(io.StringIO(data_string))

In [13]:
df_stn_map

,STN,STN_ko
0,90,속초
1,93,북춘천
2,95,철원
3,98,동두천
4,99,파주
5,100,대관령
6,101,춘천
7,102,백령도
8,104,북강릉
9,105,강릉


In [14]:
merged_df = pd.merge(
    merged_ssn_nd, 
    df_stn_map, 
    on='STN', 
    how='left',
    suffixes=('_ID', '_MD')
)

merged_df

,YY,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko
0,1974,90,1974-11-03,401,401,서리,기후,첫(강·하천 결빙),기후,속초
1,1974,90,1975-03-21,401,402,서리,기후,마지막(강·하천 해빙),기후,속초
2,1974,90,1975-03-31,402,402,얼음,기후,마지막(강·하천 해빙),기후,속초
3,1974,90,1974-12-03,403,401,눈,기후,첫(강·하천 결빙),기후,속초
4,1974,90,1975-03-23,403,402,눈,기후,마지막(강·하천 해빙),기후,속초
...,...,...,...,...,...,...,...,...,...,...
111692,2025,247,2025-04-25,503,501,군락단지,식물,시작(해수욕장 개장),생활,남원
111693,2025,247,2025-05-02,503,504,군락단지,식물,NaN,NaN,남원
111694,2025,272,2025-10-20,501,501,유명산 단풍,식물,시작(해수욕장 개장),생활,영주
111695,2025,279,2025-10-27,501,501,유명산 단풍,식물,시작(해수욕장 개장),생활,구미


### 2-4) 필요정보 결합 후, 데이터 기본 검토

In [15]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111697 entries, 0 to 111696
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   YY        111697 non-null  int64         
 1   STN       111697 non-null  int64         
 2   TM        111697 non-null  datetime64[ns]
 3   SSN_ID    111697 non-null  int64         
 4   SSN_MD    111697 non-null  int64         
 5   ID_Name   111695 non-null  object        
 6   ID_Class  111695 non-null  object        
 7   MD_Name   111500 non-null  object        
 8   MD_Class  111500 non-null  object        
 9   STN_ko    107573 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 8.5+ MB


In [16]:
merged_df.isnull().sum()

YY             0
STN            0
TM             0
SSN_ID         0
SSN_MD         0
ID_Name        2
ID_Class       2
MD_Name      197
MD_Class     197
STN_ko      4124
dtype: int64

In [17]:
merged_df.duplicated().sum()

np.int64(0)

### 2-5) 결측치 데이터 상세확인

#### ① 계절관측(SSN_ID) & 계절현상(SSN_MD) 관련 데이터

In [18]:
# SSN_ID & SSN_MD 결측치 df 생성
null_rows_df = merged_df[merged_df['ID_Name'].isnull() | merged_df['MD_Name'].isnull()].copy()

In [19]:
# 결측치 데이터 중복제거 데이터 확인
null_rows_df[['SSN_ID','SSN_MD','ID_Name', 'MD_Name']].drop_duplicates()

,SSN_ID,SSN_MD,ID_Name,MD_Name
96661,503,504,군락단지,NaN
99297,703,401,NaN,첫(강·하천 결빙)
107046,708,401,NaN,첫(강·하천 결빙)


In [20]:
# 결측치 데이터 분포 연도 확인
null_rows_df[['YY','SSN_ID','SSN_MD','ID_Name', 'MD_Name']].drop_duplicates()

,YY,SSN_ID,SSN_MD,ID_Name,MD_Name
96661,2012,503,504,군락단지,NaN
99297,2014,703,401,NaN,첫(강·하천 결빙)
99472,2014,503,504,군락단지,NaN
101023,2015,503,504,군락단지,NaN
102366,2016,503,504,군락단지,NaN
103306,2017,503,504,군락단지,NaN
104247,2018,503,504,군락단지,NaN
105187,2019,503,504,군락단지,NaN
106128,2020,503,504,군락단지,NaN
107046,2020,708,401,NaN,첫(강·하천 결빙)


In [21]:
# 결측치 데이터 분포 월 확인
null_rows_df['TM_Month'] = null_rows_df['TM'].dt.month
null_rows_df[['TM_Month','SSN_ID','SSN_MD','ID_Name', 'MD_Name']].drop_duplicates()

,TM_Month,SSN_ID,SSN_MD,ID_Name,MD_Name
96661,4,503,504,군락단지,NaN
99297,10,703,401,NaN,첫(강·하천 결빙)
101558,5,503,504,군락단지,NaN
101796,3,503,504,군락단지,NaN
101969,2,503,504,군락단지,NaN
102107,6,503,504,군락단지,NaN
107046,2,708,401,NaN,첫(강·하천 결빙)


#### ② 관측지점(STN) 관련 데이터

In [22]:
#  'STN_ko' 컬럼 결측치를 가지는 STN 중복제거 후 목록 확인
null_rows_df = merged_df[merged_df['STN_ko'].isnull()].copy()
null_rows_df[['STN']].drop_duplicates()

,STN
200,214
248,256
270,265
41068,164
63393,270
68142,175
86033,187
98752,176
99206,52


In [23]:
#  'STN_ko' 컬럼 결측치를 가지는 STN 유니크 값 별 레코드 수 확인
null_rows_df[['STN']].value_counts()

STN
256    1465
265     952
214     821
175     462
164     281
176      66
187      44
270      26
52        7
Name: count, dtype: int64

In [24]:
# 'STN_ko' 컬럼 결측치를 가지는 STN 분포 연도 확인
null_rows_df[['YY']].drop_duplicates()

,YY
200,1974
1406,1975
3250,1976
5210,1977
7232,1978
9314,1979
11434,1980
13508,1981
15502,1982
17416,1983


## 3. 결측데이터 및 'YY'컬럼 삭제

### 3-1) 결측치 데이터 삭제

In [25]:
# 1. 원본 merged_df의 크기 저장
original_count = len(merged_df)

# 2. NaN 값이 있는 모든 행을 삭제 (모든 컬럼을 검사)
# how='any'가 기본값으로, 단 하나의 NaN이라도 포함하면 해당 행을 삭제
merged_df_dropped_na = merged_df.dropna(how='any').copy()

# 3. 삭제된 레코드 수 및 최종 크기 확인
dropped_count = original_count - len(merged_df_dropped_na)
final_count = len(merged_df_dropped_na)

print("--- 🗑️ Null 값 포함 레코드 삭제 결과 ---")
print(f"원본 레코드 수: {original_count}개")
print(f"삭제된 Null 포함 레코드 수: {dropped_count}개")
print(f"최종 레코드 수: {final_count}개")

print("\n--- Null 값 삭제 후 데이터프레임 정보 ---")
print(merged_df_dropped_na.info())

--- 🗑️ Null 값 포함 레코드 삭제 결과 ---
원본 레코드 수: 111697개
삭제된 Null 포함 레코드 수: 4323개
최종 레코드 수: 107374개

--- Null 값 삭제 후 데이터프레임 정보 ---
<class 'pandas.core.frame.DataFrame'>
Index: 107374 entries, 0 to 111696
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   YY        107374 non-null  int64         
 1   STN       107374 non-null  int64         
 2   TM        107374 non-null  datetime64[ns]
 3   SSN_ID    107374 non-null  int64         
 4   SSN_MD    107374 non-null  int64         
 5   ID_Name   107374 non-null  object        
 6   ID_Class  107374 non-null  object        
 7   MD_Name   107374 non-null  object        
 8   MD_Class  107374 non-null  object        
 9   STN_ko    107374 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 9.0+ MB
None


### 3-2) 'YY' 컬럼 필요여부 확인 및 삭제

- 데이터 검토 중 TM 컬럼의 연도 값과 YY 컬럼 값이 일치하지 않는 데이터 일부 확인 후, 본 과정 진행

#### ① 'YY' 컬럼값과 'TM'의 연도 값 일치여부 확인

In [26]:
merged_df_dropped_na_year = merged_df_dropped_na['TM'].dt.year
mismatched_year_records_df = merged_df_dropped_na[merged_df_dropped_na['YY'] != merged_df_dropped_na_year].copy()
mismatched_year_records_df

,YY,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko
1,1974,90,1975-03-21,401,402,서리,기후,마지막(강·하천 해빙),기후,속초
2,1974,90,1975-03-31,402,402,얼음,기후,마지막(강·하천 해빙),기후,속초
4,1974,90,1975-03-23,403,402,눈,기후,마지막(강·하천 해빙),기후,속초
5,1974,100,1975-05-17,401,402,서리,기후,마지막(강·하천 해빙),기후,대관령
6,1974,100,1975-05-17,402,402,얼음,기후,마지막(강·하천 해빙),기후,대관령
...,...,...,...,...,...,...,...,...,...,...
110942,2024,184,2025-03-19,403,402,눈,기후,마지막(강·하천 해빙),기후,제주
110944,2024,184,2025-04-24,404,402,관설,기후,마지막(강·하천 해빙),기후,제주
110973,2024,189,2025-01-09,402,401,얼음,기후,첫(강·하천 결빙),기후,서귀포
110974,2024,189,2025-02-07,402,402,얼음,기후,마지막(강·하천 해빙),기후,서귀포


#### ② 'YY' 컬럼값과 'TM'의 연도 값 차이정도 확인

In [27]:
mismatched_year_records_df['TM_Year'] = mismatched_year_records_df['TM'].dt.year
mismatched_year_records_df['TM_Month'] = mismatched_year_records_df['TM'].dt.month

mismatched_year_records_df

,YY,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month
1,1974,90,1975-03-21,401,402,서리,기후,마지막(강·하천 해빙),기후,속초,1975,3
2,1974,90,1975-03-31,402,402,얼음,기후,마지막(강·하천 해빙),기후,속초,1975,3
4,1974,90,1975-03-23,403,402,눈,기후,마지막(강·하천 해빙),기후,속초,1975,3
5,1974,100,1975-05-17,401,402,서리,기후,마지막(강·하천 해빙),기후,대관령,1975,5
6,1974,100,1975-05-17,402,402,얼음,기후,마지막(강·하천 해빙),기후,대관령,1975,5
...,...,...,...,...,...,...,...,...,...,...,...,...
110942,2024,184,2025-03-19,403,402,눈,기후,마지막(강·하천 해빙),기후,제주,2025,3
110944,2024,184,2025-04-24,404,402,관설,기후,마지막(강·하천 해빙),기후,제주,2025,4
110973,2024,189,2025-01-09,402,401,얼음,기후,첫(강·하천 결빙),기후,서귀포,2025,1
110974,2024,189,2025-02-07,402,402,얼음,기후,마지막(강·하천 해빙),기후,서귀포,2025,2


In [ ]:
# TM-YEAR - YEAR 가 1이 아닌 레코드 추출
# 극한의 예외치 확인

year_difference_non_abs = mismatched_year_records_df['TM_Year'] - mismatched_year_records_df['YY']
non_one_difference_df = mismatched_year_records_df[year_difference_non_abs != 1].copy()
non_one_difference_df

,YY,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month
15647,1982,235,1993-01-09,405,401,강하전,기후,첫(강·하천 결빙),기후,보령,1993,1
23100,1986,119,1992-02-23,405,401,강하전,기후,첫(강·하천 결빙),기후,수원,1992,2
23101,1986,119,1992-02-25,405,402,강하전,기후,마지막(강·하천 해빙),기후,수원,1992,2
31971,1989,211,1988-12-28,405,401,강하전,기후,첫(강·하천 결빙),기후,인제,1988,12


#### ③ YY 컬럼제거
- 위 분석결과를 봤을 때, YY와 TM의 연도가 다르다는 부분을 제외하곤 크게 이상한 부분이 없는거 같아, YY 컬럼만 제거

In [29]:
merged_df_dropped_yy = merged_df_dropped_na.drop(columns=['YY'])
merged_df_dropped_yy

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko
0,90,1974-11-03,401,401,서리,기후,첫(강·하천 결빙),기후,속초
1,90,1975-03-21,401,402,서리,기후,마지막(강·하천 해빙),기후,속초
2,90,1975-03-31,402,402,얼음,기후,마지막(강·하천 해빙),기후,속초
3,90,1974-12-03,403,401,눈,기후,첫(강·하천 결빙),기후,속초
4,90,1975-03-23,403,402,눈,기후,마지막(강·하천 해빙),기후,속초
...,...,...,...,...,...,...,...,...,...
111691,226,2025-10-28,501,501,유명산 단풍,식물,시작(해수욕장 개장),생활,보은
111692,247,2025-04-25,503,501,군락단지,식물,시작(해수욕장 개장),생활,남원
111694,272,2025-10-20,501,501,유명산 단풍,식물,시작(해수욕장 개장),생활,영주
111695,279,2025-10-27,501,501,유명산 단풍,식물,시작(해수욕장 개장),생활,구미


## 4. 분석데이터 유효범위 산정

### 4-1) 지상관측지점

#### ① 관측지점별 레코드수 확인

In [30]:
def get_top_n_stn_stats(df, n=10):

    # 1. STN과 STN_ko 컬럼을 조합하여 새로운 임시 시리즈의 Index 생성
    if 'STN' not in df.columns or 'STN_ko' not in df.columns:
        print("오류: DataFrame에 'STN' 또는 'STN_ko' 컬럼이 존재하지 않습니다.")
        return pd.DataFrame()

    stn_series_index = df.set_index(['STN', 'STN_ko']).index

    # 2. value_counts를 사용하여 Count와 Percentage 계산
    # Count 계산
    stn_counts = stn_series_index.value_counts(normalize=False).rename('Count')

    # 비율 계산
    stn_percentages = stn_series_index.value_counts(normalize=True).rename('Percentage') * 100

    # 3. 결과 합치기, 정렬 및 상위 N개 추출
    # Count와 Percentage 결과를 DataFrame으로 합치고 인덱스 리셋
    stn_results = pd.concat([stn_counts, stn_percentages], axis=1).reset_index()

    # 개수가 많은 순서대로 상위 N개 추출
    top_n_stn = stn_results.sort_values(by='Count', ascending=False).head(n)
    
    # 4. 출력 포맷팅 및 결과 반환
    top_n_stn['Percentage'] = top_n_stn['Percentage'].apply(lambda x: f"{x:.2f}%")
    return top_n_stn

In [31]:
# 전체데이터에서 상위 10개 지점 확인
get_top_n_stn_stats(merged_df_dropped_yy, 10)

,STN,STN_ko,Count,Percentage
0,146,전주,2035,1.90%
1,138,포항,2021,1.88%
2,156,광주,2013,1.87%
3,108,서울,2002,1.86%
4,143,대구,1999,1.86%
5,136,안동,1983,1.85%
6,131,청주,1940,1.81%
7,152,울산,1926,1.79%
8,168,여수,1918,1.79%
9,133,대전,1915,1.78%


In [32]:
# 관측 기간을 줄여 상위 15개 지점 확인
start_date = '2009-01-01'
temp_df = merged_df_dropped_yy[merged_df_dropped_yy['TM'] > start_date].copy()

get_top_n_stn_stats(temp_df, 15)

,STN,STN_ko,Count,Percentage
0,136,안동,717,3.35%
1,104,북강릉,703,3.29%
2,146,전주,697,3.26%
3,143,대구,697,3.26%
4,138,포항,689,3.22%
5,108,서울,680,3.18%
6,155,창원,680,3.18%
7,156,광주,674,3.15%
8,133,대전,669,3.13%
9,119,수원,655,3.06%


#### ② 5개 대표 관측지점 선정
위 결과를 바탕으로, 권역별 대표 지상관측 지점 5개소 선정
- 서울
- 대전
- 북강릉
- 광주
- 울산

#### ③ 대표관측지점 관측 데이터 추출

In [33]:
# 추출할 STN_ko 목록 정의
stn_to_extract = ['서울', '대전', '북강릉', '광주', '울산']

# 1. 추출할 조건(mask) 생성: STN_ko가 목록에 포함되는 레코드 (True)
extraction_mask = merged_df_dropped_yy['STN_ko'].isin(stn_to_extract)

# 2. 마스크가 True인 레코드만 선택하여 새로운 DataFrame 생성
df_extracted_stn = merged_df_dropped_yy[extraction_mask].copy()

# 3. 추출된 레코드 수 확인
original_count = len(merged_df_dropped_yy)
extracted_count = len(df_extracted_stn)

print("--- 📥 특정 STN_ko 레코드 추출 결과 ---")
print(f"추출 대상 지역: {stn_to_extract}")
print(f"원본 레코드 수: {original_count}개")
print(f"추출된 레코드 수: {extracted_count}개")
print(f"추출 비율: {(extracted_count / original_count) * 100:.2f}%")

--- 📥 특정 STN_ko 레코드 추출 결과 ---
추출 대상 지역: ['서울', '대전', '북강릉', '광주', '울산']
원본 레코드 수: 107374개
추출된 레코드 수: 8571개
추출 비율: 7.98%


#### ④ 추출결과 데이터 검토

In [34]:
df_extracted_stn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8571 entries, 22 to 111487
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   STN       8571 non-null   int64         
 1   TM        8571 non-null   datetime64[ns]
 2   SSN_ID    8571 non-null   int64         
 3   SSN_MD    8571 non-null   int64         
 4   ID_Name   8571 non-null   object        
 5   ID_Class  8571 non-null   object        
 6   MD_Name   8571 non-null   object        
 7   MD_Class  8571 non-null   object        
 8   STN_ko    8571 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 669.6+ KB


In [35]:
df_extracted_stn.groupby('STN_ko').size()

STN_ko
광주     2013
대전     1915
북강릉     715
서울     2002
울산     1926
dtype: int64

##### 북강릉 지역에 데이터 불균형이 심함 확인 -> 전체 데이터의 관측기간 조정 필요성 확인

### 4-2) 관측일

#### ① 유효기간 산정 위해 관측 시작연도 검토
참고) '11월 1일'을 기준으로 잡은 이유 <br>
    - 본 과정은 계절별 일수 산출이 목적인 정체 과정<br>
    - 겨울의 시작 시점을 전년도 11월 부터 검토하는 방식으로 구성 예정이라 데이터 관측 시점을 '11월 1일'로 지정 

In [36]:
start_date = '2008-11-01'
temp_df = df_extracted_stn[df_extracted_stn['TM'] > start_date].copy()

In [37]:
temp_df.groupby('STN_ko').size()

STN_ko
광주     681
대전     675
북강릉    709
서울     686
울산     648
dtype: int64

##### '북강릉'지점의 전체기간 데이터 수와 '2008-11-01'이후 데이터 수 차이가 크지 않음 확인 -> '북강릉' 지점 **관측 시작일 확인** 필요 

#### ② '북강릉' 지점 관측시작일 확인

In [38]:
# 북강릉 관측소 데이터 시작시점 확인
df_extracted_stn[df_extracted_stn['STN_ko'] == '북강릉'].sort_values(by='TM').sort_values(by='TM')

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko
87678,104,2008-08-25,201,202,코스모스,식물(꽃),꽃 핌(개화),식물(꽃),북강릉
87676,104,2008-09-21,108,102,매미,동물,중견,동물,북강릉
87677,104,2008-09-21,108,104,매미,동물,종성,동물,북강릉
87681,104,2008-10-20,302,301,단풍나무,식물(단풍),단풍 시작,식물(단풍),북강릉
87675,104,2008-10-23,101,102,제비,동물,중견,동물,북강릉
...,...,...,...,...,...,...,...,...,...
111067,104,2025-10-16,201,203,코스모스,식물(꽃),활짝 핌(만발),식물(꽃),북강릉
111090,104,2025-10-27,302,301,단풍나무,식물(단풍),단풍 시작,식물(단풍),북강릉
111089,104,2025-10-28,301,301,은행나무,식물(단풍),단풍 시작,식물(단풍),북강릉
111091,104,2025-10-28,401,401,서리,기후,첫(강·하천 결빙),기후,북강릉


#### ③ 관측일 시점 '2009-11-01~' 로 규정 후, 추출

- '북강릉' 지점 관측 시작일 2008-08-25 임 확인
- 이에, 2010년도~2024년 데이터를 시각화 대상 데이터로 규정 (5년 단위로 변화 추이 시각화 예정)

In [39]:
start_date = '2009-11-01'
fin_df = df_extracted_stn[df_extracted_stn['TM'] >= start_date].copy()

#### ④ 추출결과 데이터 검토

In [40]:
fin_df

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko
90230,104,2009-11-02,401,401,서리,기후,첫(강·하천 결빙),기후,북강릉
90231,104,2010-04-08,401,402,서리,기후,마지막(강·하천 해빙),기후,북강릉
90232,104,2009-11-02,402,401,얼음,기후,첫(강·하천 결빙),기후,북강릉
90233,104,2010-04-16,402,402,얼음,기후,마지막(강·하천 해빙),기후,북강릉
90234,104,2009-11-02,403,401,눈,기후,첫(강·하천 결빙),기후,북강릉
...,...,...,...,...,...,...,...,...,...
111483,156,2025-04-02,207,203,복숭아,식물(꽃),활짝 핌(만발),식물(꽃),광주
111484,156,2025-03-24,208,201,배나무,식물(꽃),발아,식물(꽃),광주
111485,156,2025-04-07,208,202,배나무,식물(꽃),꽃 핌(개화),식물(꽃),광주
111486,156,2025-04-10,208,203,배나무,식물(꽃),활짝 핌(만발),식물(꽃),광주


In [41]:
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3215 entries, 90230 to 111487
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   STN       3215 non-null   int64         
 1   TM        3215 non-null   datetime64[ns]
 2   SSN_ID    3215 non-null   int64         
 3   SSN_MD    3215 non-null   int64         
 4   ID_Name   3215 non-null   object        
 5   ID_Class  3215 non-null   object        
 6   MD_Name   3215 non-null   object        
 7   MD_Class  3215 non-null   object        
 8   STN_ko    3215 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 251.2+ KB


##  5. 데이터 정제

### 5-1) 도메인 기반 '계절'라벨 생성

#### ① TM 컬럼에서 연, 월 데이터 추출하여 컬럼 추가
- '계절'라벨 생성 위해서는 각 관측데이터를 '월' 기준으로 검토해야 함.
- '연도' 데이터 또한 데이터 분석 및 검증 과정에서 자주 사용될 기준 데이터
- 이에 TM 컬럼에서 연, 월 데이터 추출하여 추가 컬럼 구성

In [42]:
fin_df['TM_Year'] = fin_df['TM'].dt.year
fin_df['TM_Month'] = fin_df['TM'].dt.month
fin_df.head()

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month
90230,104,2009-11-02,401,401,서리,기후,첫(강·하천 결빙),기후,북강릉,2009,11
90231,104,2010-04-08,401,402,서리,기후,마지막(강·하천 해빙),기후,북강릉,2010,4
90232,104,2009-11-02,402,401,얼음,기후,첫(강·하천 결빙),기후,북강릉,2009,11
90233,104,2010-04-16,402,402,얼음,기후,마지막(강·하천 해빙),기후,북강릉,2010,4
90234,104,2009-11-02,403,401,눈,기후,첫(강·하천 결빙),기후,북강릉,2009,11


#### ② SSN_ID & SSN_MD 유니크 조합 확인

In [43]:
unique_combination_df = fin_df[['SSN_ID', 'SSN_MD','ID_Name', 'MD_Name']].drop_duplicates()
unique_combination_df

,SSN_ID,SSN_MD,ID_Name,MD_Name
90230,401,401,서리,첫(강·하천 결빙)
90231,401,402,서리,마지막(강·하천 해빙)
90232,402,401,얼음,첫(강·하천 결빙)
90233,402,402,얼음,마지막(강·하천 해빙)
90234,403,401,눈,첫(강·하천 결빙)
90235,403,402,눈,마지막(강·하천 해빙)
90236,404,401,관설,첫(강·하천 결빙)
90237,404,402,관설,마지막(강·하천 해빙)
90238,405,401,강하전,첫(강·하천 결빙)
90239,405,402,강하전,마지막(강·하천 해빙)


#### ③ 유니크 조합이 가장 많이 관측된 TOP3 월 확인 (수량 / 비율) -> 각 '월'에 매핑된 '계절' 반환 

In [44]:
import pandas as pd

# 1. 조합별 월별 개수 계산
month_counts = (
    fin_df.groupby(['SSN_ID', 'SSN_MD', 'TM_Month'])
    .size()
    .reset_index(name='count')
)

# 2. 각 조합의 전체 대비 비율 계산
month_counts['ratio'] = (
    month_counts.groupby(['SSN_ID', 'SSN_MD'])['count']
    .transform(lambda x: x / x.sum())
)

# 3. 각 조합별 상위 3개 월 추출
top3_months = (
    month_counts
    .sort_values(['SSN_ID', 'SSN_MD', 'count'], ascending=[True, True, False])
    .groupby(['SSN_ID', 'SSN_MD'], group_keys=False)
    .head(3)
)

# 4. 월 → 계절 매핑
def month_to_season(month):
    if month in [12, 1, 2]:
        return '겨울'
    elif month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    else:
        return '가을'

top3_months['season'] = top3_months['TM_Month'].apply(month_to_season)


# 5. 우니크 조합 바탕으로 top3 월/계절 정보 상세 확인
top3_summary = (
    top3_months.groupby(['SSN_ID', 'SSN_MD'])
    .apply(lambda x: pd.Series({
        'top_months': list(x['TM_Month']),
        'month_ratios': list(x['ratio'].round(3)),
        'top_seasons': list(x['season'])
    }))
    .reset_index()
)

top3_summary


/var/folders/tg/fcv2bnfn76jc3mbk6x3mtlcr0000gn/T/ipykernel_15319/3148474901.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


,SSN_ID,SSN_MD,top_months,month_ratios,top_seasons
0,101,101,"[4, 5, 3]","[0.722, 0.25, 0.028]","[봄, 봄, 봄]"
1,101,102,[10],[1.0],[가을]
2,102,101,"[4, 5]","[0.714, 0.286]","[봄, 봄]"
3,103,101,"[4, 3]","[0.75, 0.25]","[봄, 봄]"
4,104,101,"[3, 4, 5]","[0.5, 0.388, 0.075]","[봄, 봄, 봄]"
5,105,101,"[6, 7, 5]","[0.488, 0.4, 0.088]","[여름, 여름, 봄]"
6,106,103,"[4, 5]","[0.75, 0.25]","[봄, 봄]"
7,107,101,[5],[1.0],[봄]
8,107,103,"[5, 3, 4]","[0.91, 0.03, 0.03]","[봄, 봄, 봄]"
9,107,104,[10],[1.0],[가을]


#### ④ 유니크 조합별 대표 계절 값 도출
- top3_summary 의 season_ratios를 기반으로 조합별로 대표 계절 라벨 생성

In [ ]:
# 1. 계절별 합이 가장 높은 계절 main_season으로 선정
def pick_main_season_from_ratios(row):
    season_ratios = {}
    for season, ratio in zip(row['top_seasons'], row['month_ratios']):
        season_ratios[season] = season_ratios.get(season, 0) + ratio

    # 가장 높은 합을 가진 계절 선택
    main_season = max(season_ratios, key=season_ratios.get)
    return main_season

# 2. 컬럼 추가
top3_summary['main_season'] = top3_summary.apply(pick_main_season_from_ratios, axis=1)

In [46]:
top3_summary

,SSN_ID,SSN_MD,top_months,month_ratios,top_seasons,main_season
0,101,101,"[4, 5, 3]","[0.722, 0.25, 0.028]","[봄, 봄, 봄]",봄
1,101,102,[10],[1.0],[가을],가을
2,102,101,"[4, 5]","[0.714, 0.286]","[봄, 봄]",봄
3,103,101,"[4, 3]","[0.75, 0.25]","[봄, 봄]",봄
4,104,101,"[3, 4, 5]","[0.5, 0.388, 0.075]","[봄, 봄, 봄]",봄
5,105,101,"[6, 7, 5]","[0.488, 0.4, 0.088]","[여름, 여름, 봄]",여름
6,106,103,"[4, 5]","[0.75, 0.25]","[봄, 봄]",봄
7,107,101,[5],[1.0],[봄],봄
8,107,103,"[5, 3, 4]","[0.91, 0.03, 0.03]","[봄, 봄, 봄]",봄
9,107,104,[10],[1.0],[가을],가을


In [47]:
# top_swasons 칼럼의 계절이 2개 이상 매핑된 레코드들의 결과 검증
top3_summary[
    top3_summary['top_seasons'].apply(lambda x: len(set(x)) > 1)
]

,SSN_ID,SSN_MD,top_months,month_ratios,top_seasons,main_season
5,105,101,"[6, 7, 5]","[0.488, 0.4, 0.088]","[여름, 여름, 봄]",여름
12,108,104,"[9, 10, 8]","[0.818, 0.152, 0.03]","[가을, 가을, 여름]",가을
13,109,101,"[10, 11, 12]","[0.6, 0.2, 0.2]","[가을, 가을, 겨울]",가을
15,201,202,"[9, 8, 10]","[0.75, 0.197, 0.053]","[가을, 여름, 가을]",가을
17,202,201,"[2, 3, 1]","[0.519, 0.38, 0.101]","[겨울, 봄, 겨울]",겨울
18,202,202,"[3, 2, 4]","[0.608, 0.291, 0.101]","[봄, 겨울, 봄]",봄
19,202,203,"[3, 2, 4]","[0.827, 0.135, 0.038]","[봄, 겨울, 봄]",봄
20,203,201,"[3, 2]","[0.888, 0.112]","[봄, 겨울]",봄
23,204,201,"[3, 2]","[0.912, 0.088]","[봄, 겨울]",봄
26,205,201,"[3, 4, 2]","[0.85, 0.088, 0.062]","[봄, 봄, 겨울]",봄


#### ⑤ fin_df에 main_season 라벨 추가

In [48]:
# (1) fin_df 복사본 생성 (원본 보존)
fin_df_season = fin_df.copy()

# (2) 필요한 컬럼만 남기기
mapping_df = top3_summary[['SSN_ID', 'SSN_MD', 'main_season']]

# (3) 조합 기준으로 병합 (left join)
fin_df_season = fin_df_season.merge(mapping_df, on=['SSN_ID', 'SSN_MD'], how='left')

# (4) 결과 확인
fin_df_season

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month,main_season
0,104,2009-11-02,401,401,서리,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을
1,104,2010-04-08,401,402,서리,기후,마지막(강·하천 해빙),기후,북강릉,2010,4,봄
2,104,2009-11-02,402,401,얼음,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을
3,104,2010-04-16,402,402,얼음,기후,마지막(강·하천 해빙),기후,북강릉,2010,4,봄
4,104,2009-11-02,403,401,눈,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을
...,...,...,...,...,...,...,...,...,...,...,...,...
3210,156,2025-04-02,207,203,복숭아,식물(꽃),활짝 핌(만발),식물(꽃),광주,2025,4,봄
3211,156,2025-03-24,208,201,배나무,식물(꽃),발아,식물(꽃),광주,2025,3,봄
3212,156,2025-04-07,208,202,배나무,식물(꽃),꽃 핌(개화),식물(꽃),광주,2025,4,봄
3213,156,2025-04-10,208,203,배나무,식물(꽃),활짝 핌(만발),식물(꽃),광주,2025,4,봄


### 5-2) 이상치 제거
- SSN_ID & SSN_MD 조합에 매칭된 계절과 전혀 다른 계절의 월에 관측되는 경우 이상치 데이터로 정의
- 계절간 경계구간이 있기에, 앞서 정의한 월-계절 매핑에서 앞, 뒤로 +1 구간은 정상 범위 데이터로 간주

#### ① 이상치 의심 데이터 추출

In [49]:

# 1. 분석을 위한 df 생성
analysis_df = fin_df_season[['TM_Month', 'SSN_ID', 'SSN_MD', 'ID_Name', 'MD_Name', 'main_season']].copy()

print("--- 분석용 DataFrame (analysis_df) 미리보기 ---")
print(analysis_df.head())

# 2. 계절별 정상 구간 정의
season_months = {
    '봄': [2, 3, 4, 5, 6],
    '여름': [5, 6, 7, 8, 9],
    '가을': [8, 9, 10, 11, 12],
    '겨울': [11, 12, 1, 2, 3]
}

# 3. 'main_season' 값이 정상 구간에 있는지 먼저 확인하는 함수 정의
def check_season_month_consistency(row):
    season = row['main_season']
    month = row['TM_Month']
    
    # Nan 또는 빈값 처리
    if pd.isna(season) or season == '':
        return False  # 불일치로 판단 X

    # 현재 main_season 기준 월이 season_months 범위 안에 있으면 정상(True)
    return month not in season_months.get(season, [])


# 불일치 마스크 생성 및 레코드 추출
inconsistent_mask = analysis_df.apply(check_season_month_consistency, axis=1)
inconsistent_records_df = analysis_df[inconsistent_mask].copy()

print("--- 🚨 Season과 Month가 논리적으로 충돌하는 레코드 (일부) ---")
print(inconsistent_records_df.head(10))
print(f"\n총 {len(inconsistent_records_df)}개의 논리적 불일치 레코드가 발견되었습니다.")

--- 분석용 DataFrame (analysis_df) 미리보기 ---
   TM_Month  SSN_ID  SSN_MD ID_Name       MD_Name main_season
0        11     401     401      서리    첫(강·하천 결빙)          가을
1         4     401     402      서리  마지막(강·하천 해빙)           봄
2        11     402     401      얼음    첫(강·하천 결빙)          가을
3         4     402     402      얼음  마지막(강·하천 해빙)           봄
4        11     403     401       눈    첫(강·하천 결빙)          가을
--- 🚨 Season과 Month가 논리적으로 충돌하는 레코드 (일부) ---
      TM_Month  SSN_ID  SSN_MD ID_Name       MD_Name main_season
37           1     403     401       눈    첫(강·하천 결빙)          가을
268          1     401     402      서리  마지막(강·하천 해빙)           봄
378          1     403     401       눈    첫(강·하천 결빙)          가을
556          1     403     402       눈  마지막(강·하천 해빙)           봄
558          1     404     402      관설  마지막(강·하천 해빙)           봄
888          1     404     402      관설  마지막(강·하천 해빙)           봄
1333         1     403     401       눈    첫(강·하천 결빙)          가을
1334         1     403

#### ② 이상치 데이터 중복제거 후 확인

In [50]:

inconsistent_records_df.drop_duplicates()

,TM_Month,SSN_ID,SSN_MD,ID_Name,MD_Name,main_season
37,1,403,401,눈,첫(강·하천 결빙),가을
268,1,401,402,서리,마지막(강·하천 해빙),봄
556,1,403,402,눈,마지막(강·하천 해빙),봄
558,1,404,402,관설,마지막(강·하천 해빙),봄
1510,4,105,101,잠자리,초견,여름
2383,12,403,402,눈,마지막(강·하천 해빙),봄


#### ③ 이상치 데이터 제거

In [51]:
fin_df_season_cleaned = fin_df_season.drop(index=inconsistent_records_df.index).reset_index(drop=True)
fin_df_season_cleaned

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month,main_season
0,104,2009-11-02,401,401,서리,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을
1,104,2010-04-08,401,402,서리,기후,마지막(강·하천 해빙),기후,북강릉,2010,4,봄
2,104,2009-11-02,402,401,얼음,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을
3,104,2010-04-16,402,402,얼음,기후,마지막(강·하천 해빙),기후,북강릉,2010,4,봄
4,104,2009-11-02,403,401,눈,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을
...,...,...,...,...,...,...,...,...,...,...,...,...
3192,156,2025-04-02,207,203,복숭아,식물(꽃),활짝 핌(만발),식물(꽃),광주,2025,4,봄
3193,156,2025-03-24,208,201,배나무,식물(꽃),발아,식물(꽃),광주,2025,3,봄
3194,156,2025-04-07,208,202,배나무,식물(꽃),꽃 핌(개화),식물(꽃),광주,2025,4,봄
3195,156,2025-04-10,208,203,배나무,식물(꽃),활짝 핌(만발),식물(꽃),광주,2025,4,봄


### 5-3) season_year 값 부여
- 데이터분석을 위한 season을 기준을 한 연도 값 컬럼 생성
- 'TM_Month' 값이 11, 12 and 'main_season' 값이 '겨울' 인 경우, 관측일 연도 값에서 +1 -> 다음해 겨울데이터 산출에 포함되도록 설정 
- 위 조건 외 데이터는 기존 'TM_Year' 값과 동일하게 부여

In [52]:
df_ready = fin_df_season_cleaned.copy()

# season_year 컬럼 생성
# 조건 : 11월, 12월 & 겨울
df_ready['season_year'] = df_ready.apply(
    lambda x: x['TM_Year'] + 1 
            if (x['TM_Month'] in [11, 12]) and (x['main_season'] == '겨울')
            else x['TM_Year'],
    axis=1
)

df_ready.head()

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month,main_season,season_year
0,104,2009-11-02,401,401,서리,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을,2009
1,104,2010-04-08,401,402,서리,기후,마지막(강·하천 해빙),기후,북강릉,2010,4,봄,2010
2,104,2009-11-02,402,401,얼음,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을,2009
3,104,2010-04-16,402,402,얼음,기후,마지막(강·하천 해빙),기후,북강릉,2010,4,봄,2010
4,104,2009-11-02,403,401,눈,기후,첫(강·하천 결빙),기후,북강릉,2009,11,가을,2009


## 6. 시각화 위한 데이터 유효성 검증
- S3로 업로드 전, stacked bar chart로 구현 가능한 형태로 데이터 조작이 가능한지 검증(실제로는 snowflake에서 수행 예정)
- 차트로 구현 예정인 데이터 범위 내 결측데이터 혹은 이상데이터 존재여부 검증 

### 6-1) 시각화 위한 형태로 데이터 변환

#### ① stn, season_year 로 필터링하여 각 계절의 시작일 확인

In [53]:
def get_first_dates(fin_df_season, stn, season_year):
    """
    입력: STN 번호, season_year
    출력: 봄/여름/가을/겨울의 first_date를 담은 dict
    """

    df = fin_df_season.copy()
    df["TM"] = pd.to_datetime(df["TM"])

    seasons = ["봄", "여름", "가을", "겨울"]
    result = {}

    # 해당 STN + season_year 데이터 subset
    sub = df[(df["STN"] == stn) & (df["season_year"] == season_year)]

    for s in seasons:
        if s == "겨울":
            # 겨울은 전년도 11~12월 + 해당년도 1~2월
            cond = (
                ((sub["TM"].dt.year == season_year - 1) & (sub["TM"].dt.month >= 11)) |
                ((sub["TM"].dt.year == season_year) & (sub["TM"].dt.month <= 2))
            ) & (sub["main_season"] == "겨울")

        else:
            # 봄/여름/가을은 TM.year == season_year
            cond = (sub["TM"].dt.year == season_year) & (sub["main_season"] == s)

        filtered = sub[cond]
        result[s] = filtered["TM"].min() if not filtered.empty else None

    return result


In [54]:
get_first_dates(df_ready, stn=104, season_year=2010)

{'봄': Timestamp('2010-03-21 00:00:00'),
 '여름': Timestamp('2010-06-16 00:00:00'),
 '가을': Timestamp('2010-09-04 00:00:00'),
 '겨울': Timestamp('2009-11-02 00:00:00')}

#### ② 각 계절별 일 수 및 비율 산출

In [55]:
def get_season_days(fin_df_season, stn, season_year):
    
    # 1. 계절별 시작일 추출
    fd = get_first_dates(fin_df_season, stn, season_year)
    
    # 2. 각 계절에 시작일 할당
    spring = fd["봄"]
    summer = fd["여름"]
    autumn = fd["가을"]
    winter = fd["겨울"]

    # 일자 산출 함수
    def d(a, b):
        if pd.isna(a) or pd.isna(b):
            return pd.NA
        return (b - a).days + 1  # 시작일자 포함

    # 3. 계절간 일자 산줄
    days_spring = d(spring, summer)
    days_summer = d(summer, autumn)
    days_winter = d(winter, spring)  

    # 3-1. 가을은 365 - (봄 + 여름 + 겨울)
    if pd.isna(days_spring) or pd.isna(days_summer) or pd.isna(days_winter):
        days_autumn = pd.NA
    else:
        days_autumn = 365 - (int(days_spring) + int(days_summer) + int(days_winter))

    # 4. 산출 결과 요약본 생성
    result = pd.DataFrame({
        "main_season": ["봄", "여름", "가을", "겨울"],
        "first_date": [spring, summer, autumn, winter],
        "days": [days_spring, days_summer, days_autumn, days_winter]
    })

    # 비율 계산
    result["ratio"] = result["days"].astype("Float64") / 365

    return result


In [56]:
get_season_days(df_ready, stn=104, season_year=2010)

,main_season,first_date,days,ratio
0,봄,2010-03-21,88,0.241096
1,여름,2010-06-16,81,0.221918
2,가을,2010-09-04,56,0.153425
3,겨울,2009-11-02,140,0.383562


#### [참고] 데이터 형태 변환 과정 확인

##### ① 최종 정제데이터에서 STN & season_year 값으로 필터링

In [57]:
sch_stn = 104
sch_season_year = 2023

df_ready[(df_ready['STN'] == sch_stn) & (df_ready['season_year'] == sch_season_year)].sort_values(by='TM')

,STN,TM,SSN_ID,SSN_MD,ID_Name,ID_Class,MD_Name,MD_Class,STN_ko,TM_Year,TM_Month,main_season,season_year
2453,104,2022-12-14,404,401,관설,기후,첫(강·하천 결빙),기후,북강릉,2022,12,겨울,2023
2455,104,2022-12-15,405,401,강하전,기후,첫(강·하천 결빙),기후,북강릉,2022,12,겨울,2023
2631,104,2023-01-27,202,201,매화,식물(꽃),발아,식물(꽃),북강릉,2023,1,겨울,2023
2456,104,2023-02-11,405,402,강하전,기후,마지막(강·하천 해빙),기후,북강릉,2023,2,겨울,2023
2625,104,2023-02-27,104,101,나비,동물,초견,동물,북강릉,2023,2,봄,2023
2632,104,2023-02-27,202,202,매화,식물(꽃),꽃 핌(개화),식물(꽃),북강릉,2023,2,봄,2023
2634,104,2023-03-02,203,201,개나리,식물(꽃),발아,식물(꽃),북강릉,2023,3,봄,2023
2637,104,2023-03-03,204,201,진달래,식물(꽃),발아,식물(꽃),북강릉,2023,3,봄,2023
2649,104,2023-03-06,208,201,배나무,식물(꽃),발아,식물(꽃),북강릉,2023,3,봄,2023
2633,104,2023-03-07,202,203,매화,식물(꽃),활짝 핌(만발),식물(꽃),북강릉,2023,3,봄,2023


##### ② 필터링 된 데이터에서 각 계절별 시작일 추출

In [58]:
get_first_dates(df_ready, stn=sch_stn, season_year=sch_season_year)

{'봄': Timestamp('2023-02-27 00:00:00'),
 '여름': Timestamp('2023-06-25 00:00:00'),
 '가을': Timestamp('2023-09-04 00:00:00'),
 '겨울': Timestamp('2022-12-14 00:00:00')}

##### ③ 시작일을 기반으로 각 계절 별 일수 및 비율 산출

In [59]:
get_season_days(df_ready, stn=104, season_year=2022)

,main_season,first_date,days,ratio
0,봄,2022-03-03,82,0.224658
1,여름,2022-05-23,99,0.271233
2,가을,2022-08-29,69,0.189041
3,겨울,2021-11-09,115,0.315068


### 6-2) 각 관측지점 및 계절연도 별 사계절 라벨 존재여부 확인

#### ① STN & season_year 기준으로 main_season의 고유 계절 갯수 산출

In [60]:
season_check = (
    df_ready.groupby(["STN", "season_year"])["main_season"]
    .nunique()
    .reset_index(name="season_cnt")
)

missing = season_check.query("season_cnt < 4")
missing

,STN,season_year,season_cnt
0,104,2009,1
17,108,2009,1
34,133,2009,1
51,152,2009,1
68,156,2009,1


#### ② pivot으로 missing 상세 데이터 확인 
- 상세데이터 확인 결과, 4계절 코드가 모두 존재하지 않은 연도는 2009년으로 앞서 정의한 시각화 대상 연도에서 제외된 값
- 현재 데이터로 시각화 진행 문제 없음 확인

In [61]:
season_matrix = (
    df_ready
    .pivot_table(index=["STN", "season_year"], columns="main_season", values="TM", aggfunc="count")
    .fillna(0)
    .astype(int)
)

print("=== ✅ 계절 데이터 completeness 체크 결과 ===")
display(season_matrix)

=== ✅ 계절 데이터 completeness 체크 결과 ===


main_season      가을  겨울   봄  여름
STN season_year                
104 2009          3   0   0   0
    2010         11   4  23   2
    2011         11   4  24   2
    2012         11   4  23   2
    2013         11   4  23   2
    2014         12   4  22   2
    2015          7   4  29   2
    2016          9   3  30   2
    2017          9   3  28   2
    2018         10   3  27   2
    2019          8   2  25   2
    2020          9   2  28   2
    2021          9   4  28   2
    2022          9   4  28   2
    2023          9   4  29   2
    2024          9   4  29   2
    2025          6   4  29   2
108 2009          4   0   0   0
    2010         11   4  19   2
    2011         11   4  19   2
    2012         11   4  19   2
    2013         11   4  19   2
    2014         11   4  20   2
    2015         11   4  26   2
    2016         12   4  26   2
    2017         11   4  26   2
    2018         11   4  26   2
    2019         10   4  27   2
    2020         11   2  27   2
    2021          9   4  27   2
    2022         11   2  26   2
    2023         10   4  27   2
    2024          9   4  26   2
    2025          5   4  27   2
133 2009          3   0   0   0
    2010         11   4  19   2
    2011         11   2  19   2
    2012         11   2  19   2
    2013         11   2  19   2
    2014         11   2  20   2
    2015         12   2  26   2
    2016         11   4  27   2
    2017         11   4  27   2
    2018         11   4  27   2
    2019         11   2  27   2
    2020         11   2  27   2
    2021         11   2  27   2
    2022         11   2  27   2
    2023         11   2  26   2
    2024         11   2  27   1
    2025          3   2  27   2
152 2009          4   0   0   0
    2010         12   2  20   4
    2011         11   4  20   2
    2012         10   1  19   2
    2013         12   4  18   2
    2014         11   2  20   2
    2015         11   2  18   2
    2016         10   1  25   2
    2017         11   1  24   1
    2018         11   1  25   2
    2019         10   2  26   2
    2020         11   1  25   2
    2021         11   2  27   2
    2022         11   1  24   2
    2023         11   2  26   2
    2024         11   2  27   2
    2025          3   2  24   2
156 2009          6   0   0   0
    2010         11   2  20   2
    2011         10   2  20   2
    2012         11   2  20   2
    2013         11   2  20   2
    2014         11   2  20   2
    2015         12   2  28   2
    2016         11   3  27   2
    2017         12   3  27   2
    2018         12   3  27   2
    2019         12   3  27   2
    2020         11   3  27   2
    2021         11   3  26   2
    2022         11   3  26   2
    2023         11   3  26   2
    2024         11   2  27   2
    2025          3   4  27   2